In [24]:
from tempor import plugin_loader
from data_loaders.temporAI_dataloader import load_gsu_dataset
import numpy as np

In [14]:
data_path = '/Users/jk1/temp/treatment_effects/prepro_test/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_features_25112023_213851.csv'
continuous_outcomes_path = '/Users/jk1/temp/treatment_effects/prepro_test/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_outcomes_continuous_25112023_213851.csv'

# Load GSU dataset

In [15]:
gsu_dataset = load_gsu_dataset(data_path, continuous_outcomes_path)

In [16]:
gsu_dataset

TemporalTreatmentEffectsDataset(
    time_series=TimeSeriesSamples([2493, *, 84]),
    predictive=TemporalTreatmentEffectsTaskData(
        targets=TimeSeriesSamples([2493, *, 1]),
        treatments=TimeSeriesSamples([2493, *, 1])
    )
)

## Explore model

In [19]:
temp_dataset = gsu_dataset[0:2]

In [20]:
# Initialize the model:
model = plugin_loader.get("treatments.temporal.regression.crn_regressor", epochs=1)

In [21]:
# Train the model:
model.fit(temp_dataset)

Preparing data for decoder training...
Preparing data for decoder training DONE.
=== Training stage: 1. Train encoder ===
Epoch: 0, Prediction Loss: 0.055, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 0.055
=== Training stage: 2. Train decoder ===
Epoch: 0, Prediction Loss: 0.099, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 0.099


CRNTreatmentsRegressor(
    name='crn_regressor',
    category='treatments.temporal.regression',
    plugin_type='method',
    params={
        'encoder_rnn_type': 'LSTM',
        'encoder_hidden_size': 100,
        'encoder_num_layers': 1,
        'encoder_bias': True,
        'encoder_dropout': 0.0,
        'encoder_bidirectional': False,
        'encoder_nonlinearity': None,
        'encoder_proj_size': None,
        'decoder_rnn_type': 'LSTM',
        'decoder_hidden_size': 100,
        'decoder_num_layers': 1,
        'decoder_bias': True,
        'decoder_dropout': 0.0,
        'decoder_bidirectional': False,
        'decoder_nonlinearity': None,
        'decoder_proj_size': None,
        'adapter_hidden_dims': [50],
        'adapter_out_activation': 'Tanh',
        'predictor_hidden_dims': [],
        'predictor_out_activation': None,
        'max_len': None,
        'optimizer_str': 'Adam',
        'optimizer_kwargs': {'lr': 0.01, 'weight_decay': 1e-05},
        'batch_size': 3

In [23]:
# Define horizons for each sample.
horizons = [tc.time_indexes()[0][len(tc.time_indexes()[0]) // 2 :] for tc in temp_dataset.time_series]
print("Horizons for sample 0:\n", horizons[0], end="\n\n")

Horizons for sample 0:
 [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]


In [25]:
# Define treatment scenarios for each sample.
treatment_scenarios = [[np.asarray([2] * len(h)), np.asarray([0] * len(h))] for h in horizons]
print("Alternative treatment scenarios for sample 0:\n", treatment_scenarios[0], end="\n\n")

Alternative treatment scenarios for sample 0:
 [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [41]:
# Actual treatment scenarios for sample 0:
temp_dataset.predictive.treatments[0]

TimeSeriesSamples with data:
                      anti_hypertensive_strategy
sample_idx  time_idx                            
100023_4784 0                                4.0
            1                                4.0
            2                                4.0
            3                                4.0
            4                                4.0
...                                          ...
            66                               0.0
            67                               0.0
            68                               0.0
            69                               0.0
            70                               0.0

[71 rows x 1 columns]

In [42]:
# Call predict_counterfactuals.
counterfactuals = model.predict_counterfactuals(temp_dataset, horizons=horizons, treatment_scenarios=treatment_scenarios)

In [43]:
print("Counterfactual outcomes for sample 0, given the alternative treatment scenarios:\n")
for idx, c in enumerate(counterfactuals[0]):
    print(f"Treatment scenario {idx}, {treatment_scenarios[0][idx]}")
    print(c, end="\n\n")

Counterfactual outcomes for sample 0, given the alternative treatment scenarios:

Treatment scenario 0, [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
TimeSeries() with data:
          nihss_delta_at_next_ts
time_idx                        
35                      0.061641
36                      0.055852
37                      0.055918
38                      0.055917
39                      0.055917
40                      0.055917
41                      0.055917
42                      0.055917
43                      0.055917
44                      0.055917
45                      0.055917
46                      0.055917
47                      0.055917
48                      0.055917
49                      0.055917
50                      0.055917
51                      0.055917
52                      0.055917
53                      0.055917
54                      0.055917
55                      0.055917
56                      0.055917
57    